# CH4  Naive Bayes

In [1]:
import numpy as np

## MLiA

### 文档[句子[词条]]

In [243]:
def loadDataSet():
    postingList = [['my','dog','has','flea','problems','help','please'],
                  ['maybe','not','take','him','to','dog','park','stupid'],
                  ['my','dalmation','is','so','cute','I','love','him'],
                  ['stop','posting','stupid','worthles','garbage','to','stop','him'],
                  ['quit','buying','worthless','dog','food','stupid']]
    classVec = [0,1,0,1,0,1]
    return postingList, classVec

In [248]:
postingList,classVec = loadDataSet()

### 文档->词表[List]

In [244]:
def createVocabList(dataSet):
    return list(set([y for x in dataSet for y in x]))

In [249]:
myVocabList = createVocabList(postingList)

In [250]:
myVocabList

['so',
 'posting',
 'maybe',
 'dalmation',
 'food',
 'help',
 'quit',
 'take',
 'buying',
 'please',
 'park',
 'is',
 'problems',
 'garbage',
 'stop',
 'cute',
 'worthles',
 'not',
 'to',
 'stupid',
 'worthless',
 'love',
 'I',
 'has',
 'flea',
 'him',
 'dog',
 'my']

### 句子+词表->词向量[List]

In [240]:
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

In [251]:
setOfWords2Vec(myVocabList,postingList[0])

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1]

### 词向量->概率

#### Set of Word

p(w/ci),p(ci)

In [257]:
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix) # 句子，样本数
    numWords = len(trainMatrix[0])  # 词向量长度
    pAbusive = sum(trainCategory)/float(numTrainDocs) # 类别为1的样本占比
    p0Num = np.ones(numWords);p1Num = np.ones(numWords)
    p0Denom = 2.0; p1Denom = 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]        # 向量：样本汇总，所有样本加到一个词向量长度的向量
            p1Denom += sum(trainMatrix[i]) # 标量：所有单词计数
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num/p1Denom)   # 向量，训练数据中归为1类的样本统计出的，词向量中每个元素出现的概率
    p0Vect = np.log(p0Num/p0Denom)   # 
    return p0Vect, p1Vect, pAbusive

In [258]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify*p1Vec) + np.log(pClass1)
    p0 = sum(vec2Classify*p0Vec) + np.log(1-pClass1)
    return 1 if p1 > p0 else 0 # 这里可以再处理

In [259]:
def testingNB():
    listOfPosts, listClasses = loadDataSet()
    myVocabList = createVocabList(listOfPosts)
    trainMat = []
    for postinDoc in listOfPosts:
        trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
    p0V,p1V,pAb = trainNB0(np.array(trainMat),np.array(listClasses))
    testEntry = ["love","my","dalmation"]
    thisDoc = np.array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,"classified as: ",classifyNB(thisDoc,p0Vec=p0V,p1Vec=p1V,pClass1=pAb))
    testEntry = ["stupid","garbage"]
    thisDoc = np.array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,"classified as: ",classifyNB(thisDoc,p0Vec=p0V,p1Vec=p1V,pClass1=pAb))    

In [12]:
testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


#### Bag of Word

In [252]:
def bagOfWords2VecMN(vocabList,inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
        else:
            print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

### EX1 : Spam

In [235]:
def textParse(bigString):
    import re
    listOfTokens = re.split(r'\W*',bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

In [15]:
import math
def spamTest():
    docList=[];classList=[];fullText=[]
    for i in list(range(1,26)):
        wordList = textParse(open('./Data/CH4/email/spam/%d.txt' % i,encoding="latin_1").read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('./Data/CH4/email/ham/%d.txt' % i,encoding="latin_1").read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)
    trainingSet = list(range(len(wordList)));testSet=[]
    for i in list(range(10)):
        randIndex = int(np.random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
    trainMat = [];trainClasses = []
    for docIndex in trainingSet:
        trainMat.append(bagOfWords2VecMN(vocabList,docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(np.array(trainMat),np.array(trainClasses))
    errorCount = 0
    for docIndex in testSet:
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
    print("the error rate is : ", errorCount/len(testSet))
        

In [16]:
spamTest()

D:\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


the error rate is :  0.6


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


### EX2 : Local Word

In [231]:
def calcMostFreq(vocabList,fullText):
    import operator
    freqDict = {}
    for token in vocabList:
        freqDict[token] = fullText.count(token)
#     print(freqDict)
    sortedFreq = sorted(freqDict.items(),key=lambda x:x[1],reverse=True)
    return sortedFreq[:30]

In [232]:
def localWords(feed1,feed0):
    import feedparser
    docList=[]; classList = []; fullText =[]
    minLen = min(len(feed1['entries']),len(feed0['entries']))
    for i in list(range(minLen)):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1) #NY is class 1
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)#create vocabulary
    print(len(vocabList))
    top30Words = calcMostFreq(vocabList,fullText)   #remove top 30 words
    for pairW in top30Words:
        if pairW[0] in vocabList: vocabList.remove(pairW[0])
            
    # https://www.ranks.nl/stopwords
    with open("./Data/CH4/stopwords.txt") as f:
        stopwords = f.readlines()
    stopwords = [x.strip() for x in stopwords]
    for pairW in stopwords:
        if pairW in vocabList: vocabList.remove(pairW)

    print(len(vocabList))
    trainingSet = list(range(2*minLen)); testSet=[]           #create test set
    for i in list(range(20)):
        randIndex = int(np.random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])  
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(np.array(trainMat),np.array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
    print('the error rate is: ',float(errorCount)/len(testSet))
    return vocabList,p0V,p1V

In [233]:
import feedparser
ny = feedparser.parse("https://newyork.craigslist.org/search/sof?format=rss")
sf = feedparser.parse("https://sfbay.craigslist.org/search/sof?format=rss")

In [253]:
vocabList,pSF,pNY = localWords(ny,sf)

692
632
the word: work is not in my Vocabulary!
the word: for is not in my Vocabulary!
the word: company is not in my Vocabulary!
the word: and is not in my Vocabulary!
the word: have is not in my Vocabulary!
the word: experience is not in my Vocabulary!
the word: you is not in my Vocabulary!
the word: are is not in my Vocabulary!
the word: and is not in my Vocabulary!
the word: who is not in my Vocabulary!
the word: all is not in my Vocabulary!
the word: work is not in my Vocabulary!
the word: the is not in my Vocabulary!
the word: development is not in my Vocabulary!
the word: and is not in my Vocabulary!
the word: for is not in my Vocabulary!
the word: data is not in my Vocabulary!
the word: data is not in my Vocabulary!
the word: analyst is not in my Vocabulary!
the word: who is not in my Vocabulary!
the word: are is not in my Vocabulary!
the word: the is not in my Vocabulary!
the word: only is not in my Vocabulary!
the word: and is not in my Vocabulary!
the word: services is not i

D:\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


NameError: name 'trainNB0' is not defined

In [254]:
# https://www.ranks.nl/stopwords
with open("./Data/CH4//stopwords.txt") as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]


In [255]:
def getTopWords(ny,sf):
    import operator
    vocabList,p0V,p1V=localWords(ny,sf)
    topNY=[]; topSF=[]
    for i in list(range(len(p0V))):
        if p0V[i] > -6.0 : topSF.append((vocabList[i],p0V[i]))
        if p1V[i] > -6.0 : topNY.append((vocabList[i],p1V[i]))
    sortedSF = sorted(topSF, key=lambda pair: pair[1], reverse=True)
    print("SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**")
    for item in sortedSF:
        print (item[0])
    sortedNY = sorted(topNY, key=lambda pair: pair[1], reverse=True)
    print("NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**")
    for item in sortedNY:
        print (item[0])

In [260]:
getTopWords(ny,sf)

D:\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


692
632
the word: are is not in my Vocabulary!
the word: and is not in my Vocabulary!
the word: who is not in my Vocabulary!
the word: all is not in my Vocabulary!
the word: work is not in my Vocabulary!
the word: the is not in my Vocabulary!
the word: development is not in my Vocabulary!
the word: and is not in my Vocabulary!
the word: for is not in my Vocabulary!
the word: are is not in my Vocabulary!
the word: and is not in my Vocabulary!
the word: who is not in my Vocabulary!
the word: all is not in my Vocabulary!
the word: work is not in my Vocabulary!
the word: the is not in my Vocabulary!
the word: development is not in my Vocabulary!
the word: and is not in my Vocabulary!
the word: for is not in my Vocabulary!
the word: are is not in my Vocabulary!
the word: you is not in my Vocabulary!
the word: data is not in my Vocabulary!
the word: analyst is not in my Vocabulary!
the word: that is not in my Vocabulary!
the word: the is not in my Vocabulary!
the word: being is not in my Voc

trackingtag
solving
seeking
cold
accounting
internship
general
now
shown
tester
can
graphics
combo
mateo
25011
rds
technical
clerk
independent
jersey
architectures
puzzle
dependance
content
knowledge
queens
reduce
successful
train
library
leave
must
find
providing
interested
creators
jobs
recovery
range
buy
main
programming
python
softwa
proper
computer
withgoogle
making
jupiter
electronic
years
fonts
h1b
presented
consulting
use
addition
devices
specialist
products
benefits
meet
event
document
youtube
era
quality
optical
genuine
living
resume
wbe
overcoming
imagine
present
tha
blockchain
meaningful
demo
planning
business
coordinator
interest
family
grow
island
expert
intervention
administration
human
ads
digital
investigation
firm
graphql
life
survivors
sick
location
know
make
excell
challenging
project
information
one
disability
title
hedge
class
beauty
tired
students
matlab
beach
thinker
results
delight
tra
100
long
background
stack
abuse
expand
crisis
apply
excited
stuck
learn
prov

## Sklearn

### EX1 : Spam

#### Word Vector

In [3]:
import pandas as pd
from sklearn.feature_extraction import text
import glob

In [149]:
# ham
files = "./Data/CH4/email/ham/*.txt"
ham_lst = glob.glob(files)
labels = [0]*len(ham_lst)
files = "./Data/CH4/email/spam/*.txt"
spam_lst = glob.glob(files)
lst = ham_lst + spam_lst
labels.extend([1]*len(spam_lst))
cVct = text.CountVectorizer(input="filename",encoding="latin_1")
X = cVct.fit_transform(lst).todense().A
y = labels

#### BernoulliNB

In [159]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
bnb = BernoulliNB()

In [205]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0,stratify=y)
bnb.fit(X_train,y_train)
y_pred = bnb.predict(X_test)
print("error rate is %1.2f%%" % (100*np.mean(y_pred != y_test)))

error rate is 10.00%


以上
- 生成样本的时候有顺序，训练和测试分割的时候记得stratify，不然错误率会和随机种子变化


#### Cross Validation

In [198]:
from sklearn.model_selection import cross_validate

In [208]:
result = cross_validate(X=X,y=y,cv=10,estimator=bnb)
print("test score is %1.2f" % (np.mean(result["test_score"])))

test score is 0.93


### EX2 ：Local Word

- RawData->WordList
- RawData->Vect

#### Demo

In [467]:
from sklearn.feature_extraction.text import CountVectorizer
X="smirk smirk smirk nice smirk comming comming comming back back is is a g".split(" ")
cVct = CountVectorizer(input="content")

In [468]:
result = cVct.fit_transform(X)
result.toarray()

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=int64)

In [469]:
cVct.get_feature_names()

['back', 'comming', 'is', 'nice', 'smirk']

In [470]:
cVct.vocabulary_

{'back': 0, 'comming': 1, 'is': 2, 'nice': 3, 'smirk': 4}

In [472]:
result.toarray()

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=int64)

In [464]:
def getMostFreq(cVct,X):
    tf = sum(X.toarray()).tolist()
    tf = dict(zip(cVct.get_feature_names(),tf))
    return sorted(tf.items(),key=lambda d:d[1],reverse=True)

In [473]:
getMostFreq(cVct,result)

[('smirk', 4), ('comming', 3), ('back', 2), ('is', 2), ('nice', 1)]

#### CountVectorizer

In [454]:
import feedparser
ny = feedparser.parse("https://newyork.craigslist.org/search/sof?format=rss")
sf = feedparser.parse("https://sfbay.craigslist.org/search/sof?format=rss")

In [484]:
cVct = CountVectorizer(stop_words=stopwords)

In [485]:
X = [x['summary'] for x in ny['entries']]
result = cVct.fit_transform(X)
getMostFreq(cVct,result)[:30]

[('looking', 11),
 ('software', 9),
 ('company', 8),
 ('work', 8),
 ('developer', 7),
 ('design', 6),
 ('java', 6),
 ('data', 5),
 ('applications', 4),
 ('business', 4),
 ('com', 4),
 ('develop', 4),
 ('engineer', 4),
 ('experience', 4),
 ('family', 4),
 ('qa', 4),
 ('science', 4),
 ('senior', 4),
 ('time', 4),
 ('year', 4),
 ('analyst', 3),
 ('ba', 3),
 ('based', 3),
 ('beach', 3),
 ('candidates', 3),
 ('cloud', 3),
 ('code', 3),
 ('cold', 3),
 ('computer', 3),
 ('developers', 3)]

In [486]:
X = [x['summary'] for x in sf['entries']]
result = cVct.fit_transform(X)
getMostFreq(cVct,result)[:30]

[('company', 8),
 ('developer', 6),
 ('experience', 6),
 ('full', 6),
 ('games', 6),
 ('re', 6),
 ('code', 5),
 ('craigslist', 5),
 ('design', 5),
 ('growing', 5),
 ('job', 5),
 ('millions', 5),
 ('mobile', 5),
 ('new', 5),
 ('seeks', 5),
 ('services', 5),
 ('software', 5),
 ('test', 5),
 ('california', 4),
 ('contribute', 4),
 ('daily', 4),
 ('developers', 4),
 ('development', 4),
 ('draw', 4),
 ('driven', 4),
 ('enjoy', 4),
 ('hated', 4),
 ('loved', 4),
 ('open', 4),
 ('projects', 4)]

#### TFIDF

## References

- 黑客与画家CH8
- Sklearn中有[文本分类的例子](http://scikit-learn.org/stable/auto_examples/text/document_classification_20newsgroups.html#sphx-glr-auto-examples-text-document-classification-20newsgroups-py)